# Respiratory Signal Analysis

### Import packages

In [1]:
import neurokit2 as nk
import os
import numpy as np
from scipy import signal
import csv
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib tk

### Specify data path and initialize variables

In [2]:
sampling_rate = 250.0
data_path = r'sample_data/'
signal_path = os.path.join(data_path, 'P1_S1_baseline_1674758272_855629.csv')


### Load the data and plot signal

In [3]:
resp = []
arduino_ts = []
event_code = []
skip_first = True
with open(signal_path, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for ln in csvreader:
        if skip_first:
            print(ln)
            skip_first = False
        else:
            # print(ln)
            # break
            resp.append(float(ln[1]))
            arduino_ts.append(float(ln[4]))
            if ln[5] != '':
                event_code.append(float(ln[5]))
            else:
                event_code.append(-1)

resp = np.array(resp)
arduino_ts = np.array(arduino_ts)
arduino_ts = (arduino_ts - arduino_ts[0])/1000
event_code = np.array(event_code)
# plt.plot(np.arange(0, len(raw_signal)/sampling_rate, 1/sampling_rate), raw_signal)

['EDA', 'Resp', 'PPG Finger', 'PPG Ear', 'arduino_ts', 'event_code']


In [4]:
plt.plot(arduino_ts, resp, 'g')
plt.plot(arduino_ts[event_code != -1], resp[event_code != -1], 'ro', markersize=2)
plt.xlabel("Time (seconds)")
plt.ylabel("Signal Amplitude")


Text(0, 0.5, 'Signal Amplitude')

***
### Process Respiratory Signal
***

In [5]:
sos = signal.butter(2, (0.1, 0.5), 'bandpass', fs=sampling_rate, output='sos')
filtered_resp = signal.sosfilt(sos, resp)

In [6]:
# Clean signal
cleaned = nk.rsp_clean(filtered_resp, sampling_rate=sampling_rate)

# Extract peaks
df, peaks_dict = nk.rsp_peaks(cleaned) 
info = nk.rsp_fixpeaks(peaks_dict)
formatted = nk.signal_formatpeaks(info, desired_length=len(cleaned),peak_indices=info["RSP_Peaks"])

In [7]:
fixed_peaks = nk.events_plot(info['RSP_Peaks'], cleaned)

In [8]:
# Extract rate
rsp_rate = nk.rsp_rate(cleaned, peaks_dict, sampling_rate=sampling_rate)

# Visualize
nk.signal_plot(rsp_rate, sampling_rate=sampling_rate)
plt.ylabel('Breaths Per Minute')

Text(0, 0.5, 'Breaths Per Minute')

In [9]:
rrv = nk.rsp_rrv(rsp_rate, info, sampling_rate=sampling_rate, show=True)
rrv

,RRV_RMSSD,RRV_MeanBB,RRV_SDBB,RRV_SDSD,RRV_CVBB,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_VLF,RRV_LF,RRV_HF,RRV_LFHF,RRV_LFn,RRV_HFn,RRV_SD1,RRV_SD2,RRV_SD2SD1,RRV_ApEn,RRV_SampEn
0,2056.354379,4292.16,1881.500989,2075.099467,0.438358,0.479095,3804.0,1375.8528,0.361686,NaN,0.016264,0.000414,39.290136,NaN,NaN,1467.316905,2219.701117,1.512762,0.20778,1.609438
